In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import datasets

In [121]:
x1 = tf.Variable(3, name="x1")
y1 = tf.Variable(4, name="y1")
f = x1*y1 + x1 + 10
with tf.Session() as sess:
    x1.initializer.run()
    y1.initializer.run()
    result = f.eval()
print result

25


In [2]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [115]:
boston = datasets.load_boston()

data, target = boston.data, boston.target
m, n = data.shape
design_matrix = np.c_[np.ones((m, 1)), data]

X = tf.constant(design_matrix, dtype=tf.float64, name="X")
y = tf.constant(target.reshape(-1, 1), dtype=tf.float64, name="y")
X_transpose = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_transpose, X)), X_transpose), y)

with tf.Session() as sess:
    theta_vals = theta.eval()
tf_theta = theta_vals[theta_vals[:, 0].argsort()]

In [119]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(data, target)
coef = regr.coef_[regr.coef_.argsort()]
sk_theta = np.append(coef, regr.intercept_)

for thetas in zip(np.ndarray.tolist(tf_theta[:, 0]), np.ndarray.tolist(sk_theta)):
    print thetas, thetas[0] - thetas[1]

(-17.795758660307452, -17.79575866030896) 1.50635059981e-12
(-1.4757587965195165, -1.4757587965198167) 3.00204305859e-13
(-0.9534635546900199, -0.9534635546905583) 5.38458166943e-13
(-0.5254666329007316, -0.5254666329007893) 5.77315972805e-14
(-0.10717055656036965, -0.10717055656035566) -1.39888101103e-14
(-0.012329346305273636, -0.01232934630527538) 1.74339709336e-15
(0.0007510617033220923, 0.000751061703319067) 3.0253577421e-15
(0.00939251272219142, 0.009392512722190105) 1.31492039479e-15
(0.0208602395323135, 0.02086023953217235) 1.41150979793e-13
(0.046395219529808476, 0.046395219529801954) 6.52256026967e-15
(0.3056550383390898, 0.30565503833909946) -9.65894031424e-15
(2.688561399318755, 2.6885613993179542) 8.0069284536e-13
(3.804752460259593, 3.804752460258008) 1.58495438995e-12
(36.49110328033986, 36.491103280363134) -2.32773800235e-11


In [72]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [84]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9023
